<center><h3>Model</h3></center>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
data_purchases = pd.read_csv(r"credit_card_purchases.csv")

In [3]:
data_purchases['trans_date_trans_time'] = pd.to_datetime(data_purchases['trans_date_trans_time'], errors='coerce')
data_purchases['hour'] = data_purchases['trans_date_trans_time'].dt.hour

In [4]:
data_purchases['minute'] = data_purchases['trans_date_trans_time'].dt.minute

In [5]:
data_purchases.drop(columns=['Unnamed: 0', 'first','last','street','cc_num','cc_num','trans_num','city_pop','zip','unix_time','merch_zipcode','merch_long','merch_lat','trans_date_trans_time','merchant','gender','city','state','lat','long','job','dob'], inplace=True)

In [6]:
category_preprocc = LabelEncoder()
category_transformed = category_preprocc.fit_transform(data_purchases['category'])

amt_min_max = MinMaxScaler()
amt_transformed = amt_min_max.fit_transform(np.array(data_purchases['amt']).reshape(-1,1))

data_purchases['category'] = category_transformed
data_purchases['amt'] = amt_transformed

In [7]:
# Step 1: Separar las clases mayoritaria y minoritaria
class_0 = data_purchases[data_purchases['is_fraud'] == 0]
class_1 = data_purchases[data_purchases['is_fraud'] == 1]

# Step 2: Oversampling the minority class
class_1_over = class_1.sample(n=len(class_1), replace=True, random_state=42)

# Step 3: Combinar las clases mayoritaria y sobremuestreada
balanced_data = pd.concat([class_0, class_1_over])

# Randomly shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=0.01, random_state=42).reset_index(drop=True)

# Step 4: Separate the predictor variables (X) and the target variable (y) after balancing.
X = balanced_data.drop(columns=['is_fraud'])
y = balanced_data['is_fraud']

# Step 5: Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Create and train the SVC model
svc_model = SVC(kernel='linear')
svc_model.fit(X_train, y_train)

# Step 8: Making predictions on the test set
y_pred = svc_model.predict(X_test)

accuracy = accuracy_score(svc_model.predict(data_purchases.drop(columns='is_fraud')), data_purchases['is_fraud'])

print(f"Model accuracy: {accuracy:.4f}")


Model accuracy: 0.9942
